<a href="https://colab.research.google.com/github/garlic3/ai/blob/main/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89(%EC%A0%95%EC%A0%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 구글 드라이브 연결

from google.colab import drive
drive.mount('/gdrive')
drivepath = '/gdrive/MyDrive/Colab Notebooks/titanic/'

Mounted at /gdrive


In [3]:
import pandas as pd
train = pd.read_csv(drivepath + 'train.csv')

train.head()
test = pd.read_csv(drivepath + 'test.csv')
submission = pd.read_csv(drivepath + 'submission.csv')

print(train.shape, test.shape, submission.shape)

(891, 12) (418, 11) (418, 2)


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
test.head()
# 목표변수 survived 가 없음

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [6]:
submission.head()
# 예측 값을 survived에 입력하여 제출

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [7]:
# train 데이터 프레임의 통계 정보
train.describe()
# pcalss = 선실 등급
# sibsp = 형제자매
# parch = 부모자식
# fare = 티켓 요금
# 기본적으로 숫자 데이터만 정보 분석

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
# 상관 관계 분석
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [9]:
test.corr()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.026751,-0.034102,0.003818,0.043080,0.008211
Pclass,-0.026751,1.000000,-0.492143,0.001087,0.018721,-0.577147
Age,-0.034102,-0.492143,1.000000,-0.091587,-0.061249,0.337932
SibSp,0.003818,0.001087,-0.091587,1.000000,0.306895,0.171539
Parch,0.043080,0.018721,-0.061249,0.306895,1.000000,0.230046
Fare,0.008211,-0.577147,0.337932,0.171539,0.230046,1.000000


In [10]:
# 타이타닉 전체 데이터셋 병합
train['TrainSplit'] = 'Train'
test['TrainSplit'] = 'Test'
data = pd.concat([train, test], axis = 0)
data.shape

(1309, 13)

In [11]:
# 성별을 숫자 데이터로 변경하기
data.loc[data['Sex']== 'female','Sex'] = int(0)
data.loc[data['Sex']== 'male', 'Sex'] = int(1)
data['Sex'].value_counts()


1    843
0    466
Name: Sex, dtype: int64

In [12]:
# 성별 데이터 타입 object에서 int로 변환
data['Sex'] = data['Sex'].astype('int64')
data['Sex'].dtype

dtype('int64')

In [13]:
# 숫자형 피처 값들만 추출
data_num = data.loc[:,['Sex','Pclass','Age','SibSp','Parch','Fare','Survived']]
data_num

,Sex,Pclass,Age,SibSp,Parch,Fare,Survived
0,1,3,22.0,1,0,7.2500,0.0
1,0,1,38.0,1,0,71.2833,1.0
2,0,3,26.0,0,0,7.9250,1.0
3,0,1,35.0,1,0,53.1000,1.0
4,1,3,35.0,0,0,8.0500,0.0
...,...,...,...,...,...,...,...
413,1,3,NaN,0,0,8.0500,NaN
414,0,1,39.0,0,0,108.9000,NaN
415,1,3,38.5,0,0,7.2500,NaN
416,1,3,NaN,0,0,8.0500,NaN


In [14]:
# 결측값 대체
data_num['Age'] = data_num['Age'].fillna(data_num['Age'].mean())
# 결측치를 나이의 평균으로 처리
data_num['Fare'] = data_num['Fare'].fillna(data_num['Fare'].mode()[0])
# 결측치를 요금의 최빈값으로 처리

In [15]:
# 숫자 데이터들 결측치 확인
data_num.isnull().sum() 

Sex           0
Pclass        0
Age           0
SibSp         0
Parch         0
Fare          0
Survived    418
dtype: int64

In [16]:
# 먼저 df 을 문제지와 답안지 형태로 다시 분할한다. 
# 문제지
x_data = data_num.loc[:,'Sex':'Fare']

# 답안지
y_data = data_num.loc[:,'Survived']

In [17]:
# 학습용 데이터와 검증용 데이터로 구분

X_train = data_num.loc[data['TrainSplit']=='Train',['Sex','Pclass','Age','SibSp','Parch','Fare']]
y_train = data_num.loc[data['TrainSplit']=='Train','Survived']
X_test = data_num.loc[data['TrainSplit']=='Test',['Sex','Pclass','Age','SibSp','Parch','Fare']]


In [18]:
# 데이터를 80:20로 분류
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.2, shuffle=True, random_state=20)

In [24]:
# knn 알고리즘
from sklearn.neighbors import KNeighborsClassifier  # KNN 불러오기
from sklearn.metrics import accuracy_score
r_list = []
s = 1
for i in range(1,37):
  knn = KNeighborsClassifier(n_neighbors = i) # 최적의 k값 찾기 -> 1부터 루트 k까지 시도해볼것
# 학습
  knn.fit(X_tr, y_tr)
# 예측
  t_knn_pred = knn.predict(X_val)
# 평가
  knn_acc = accuracy_score(y_val, t_knn_pred)
  r_list.append([s,knn_acc])
  s += 1
print(r_list) 

[[1, 0.7150837988826816], [2, 0.7150837988826816], [3, 0.7541899441340782], [4, 0.7597765363128491], [5, 0.7318435754189944], [6, 0.7094972067039106], [7, 0.7150837988826816], [8, 0.6703910614525139], [9, 0.7206703910614525], [10, 0.6871508379888268], [11, 0.7094972067039106], [12, 0.7039106145251397], [13, 0.7150837988826816], [14, 0.7039106145251397], [15, 0.7094972067039106], [16, 0.7150837988826816], [17, 0.7039106145251397], [18, 0.7150837988826816], [19, 0.7150837988826816], [20, 0.7150837988826816], [21, 0.7094972067039106], [22, 0.7262569832402235], [23, 0.7318435754189944], [24, 0.7430167597765364], [25, 0.7262569832402235], [26, 0.7318435754189944], [27, 0.7094972067039106], [28, 0.7206703910614525], [29, 0.7094972067039106], [30, 0.7094972067039106], [31, 0.7150837988826816], [32, 0.7206703910614525], [33, 0.7262569832402235], [34, 0.7150837988826816], [35, 0.7206703910614525], [36, 0.7150837988826816]]


In [25]:
# Knn 알고리즘
import pandas as pd

df = pd.DataFrame(r_list, columns = ['k값','결과'])
df

,k값,결과
0,1,0.715084
1,2,0.715084
2,3,0.754190
3,4,0.759777
4,5,0.731844
5,6,0.709497
6,7,0.715084
7,8,0.670391
8,9,0.720670
9,10,0.687151


In [26]:
# svm
from sklearn.svm import SVC
svc = SVC(kernel='rbf') 

# 학습
svc.fit(X_tr, y_tr)

# 예측
y_svc_pred = svc.predict(X_val)

# 평가
svc_acc = accuracy_score(y_val, y_svc_pred)
# rbf = 0.6871508379888268
# poly = 0.6703910614525139
# linear = 0.8324022346368715
svc_acc

0.6871508379888268

In [27]:
# decision tree

d_list = []
from sklearn.tree import DecisionTreeClassifier

for i in range(1,6):
  dtc = DecisionTreeClassifier(max_depth = i, random_state=20)
# 학습
  dtc.fit(X_tr, y_tr)
# 예측
  y_dtc_pred = dtc.predict(X_val)
# 평가
  dtc_acc = accuracy_score(y_val, y_dtc_pred)
  d_list.append([i,dtc_acc])
df = pd.DataFrame(d_list, columns = ['깊이값','결과'])
df

,깊이값,결과
0,1,0.832402
1,2,0.787709
2,3,0.860335
3,4,0.815642
4,5,0.821229


In [28]:
# ensemble - voting
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

hvc = VotingClassifier(estimators = [('KNN',knn),('SVM',svc),('DT',dtc)], voting='hard')

# 학습
hvc.fit(X_tr, y_tr)

# 예측
y_hvc_pred = hvc.predict(X_val)

# 평가
hvc_acc = accuracy_score(y_val, y_hvc_pred)
hvc_acc

0.7262569832402235

In [29]:
# ensemble - bagging
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators= 50, max_depth=10, random_state=20)

#학습
rfc.fit(X_tr, y_tr)

# 예측
y_rfc_pred = rfc.predict(X_val)

# 평가
rfc_acc = accuracy_score(y_val, y_rfc_pred)
rfc_acc
# n = 100 / 0.8491620111731844
# n = 50 / 0.8547486033519553

0.8547486033519553

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


xgbc = XGBClassifier(n_estimators=50, max_depth=3, random_state=20)

#학습
xgbc.fit(X_tr, y_tr) 

#예측
y_xgbc_pred = xgbc.predict(X_val)

#평가
xgbc_acc = accuracy_score(y_val, y_xgbc_pred)
xgbc_acc


0.8268156424581006